In [7]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import time

# Setting pandas to display columns
pd.set_option('display.max_columns', None)

In [2]:
nfl_small2_end_of_drive = pd.read_csv('nfl_small_end_of_drive.csv',index_col=1,\
    dtype= {'ARI' : 'str','ATL' : 'str', 'BAL' : 'str', 'BUF' : 'str', 'CAR' : 'str',
             'CHI' : 'str', 'CIN' : 'str', 'CLE' : 'str', 'DAL' : 'str', 'DEN' : 'str',
             'DET' : 'str', 'GB' : 'str', 'HOU' : 'str', 'IND' : 'str', 'JAX' : 'str',
             'KC' : 'str', 'LA' : 'str', 'LAC' : 'str', 'MIA' : 'str', 'MIN' : 'str',
             'NE' : 'str', 'NO' : 'str', 'NYG' : 'str','NYJ' : 'str', 'OAK' : 'str',
             'PHI' : 'str', 'PIT' : 'str','SEA' : 'str', 'SF' : 'str', 'TB' : 'str',
             'TEN' : 'str', 'WAS' : 'str'})

In [3]:
# Creating Home and Away Win Columns
nfl_small2_end_of_drive['home_team_win'] = np.where(nfl_small2_end_of_drive['total_home_score']\
                              > nfl_small2_end_of_drive['total_away_score'],1,0)

nfl_small2_end_of_drive['away_team_win'] = np.where(nfl_small2_end_of_drive['total_home_score']\
                              < nfl_small2_end_of_drive['total_away_score'],1,0)

In [4]:
# Getting teams to add team matrix for easier filtering
teams = list(nfl_small2_end_of_drive.groupby('posteam').sum().index)
teams[:4]

['ARI', 'ATL', 'BAL', 'BUF']

In [8]:
nfl_small2_end_of_drive.head(2)

,index,game_id,game_date,time,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,qtr,home_team,away_team,posteam,posteam_type,defteam,side_of_field,yardline_100,drive,sp,down,goal_to_go,yrdln,ydstogo,ydsnet,desc,play_type,yards_gained,pass_length,air_yards,yards_after_catch,field_goal_result,kick_distance,extra_point_result,two_point_conv_result,td_team,total_home_score,total_away_score,posteam_score,defteam_score,sack,touchdown,pass_touchdown,rush_touchdown,return_touchdown,extra_point_attempt,two_point_attempt,field_goal_attempt,fumble,complete_pass,Start_Time,year,diff,Estimated_Time,Estimated Time (EST),Estimated_Hour,Air Pressure (hPa),City,Dewpoint (°C),Field,Humidity (%),Precipitation (mm),Roof,Team Abbreviation,Temperature (°C),Time (EST),Time (GMT),Unnamed: 0_y,Wind Direction (deg),Wind Speed (km/h),added_time,game_name,end_of_drive,end_of_game,INT,ARI,ATL,BAL,BUF,CAR,CHI,CIN,CLE,DAL,DEN,DET,GB,HOU,IND,JAX,KC,LA,LAC,MIA,MIN,NE,NO,NYG,NYJ,OAK,PHI,PIT,SEA,SF,TB,TEN,WAS,week,home_team_win,away_team_win
play_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
36,270409,2015091000,2015-09-10 00:00:00,15:00,900.0,1800.0,3600.0,Half1,0,1,NE,PIT,PIT,away,NE,NE,35.0,1,0,NaN,0.0,NE 35,0,0,S.Gostkowski kicks 65 yards from NE 35 to end ...,kickoff,0.0,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-09-10 20:40:00,2015.0,0 days 00:00:00.000000000,2015-09-10 20:40:00,2015-09-10 20:40:00,2015-09-10 21:00:00,1010.6,Foxborough,16.1,FieldTurf CORE,87.0,0.3,Open,NE,18.3,2015-09-10 21:00:00,2015-09-11 01:00:00,NaN,30.0,20.5,0,PIT @ NE 2015-09-10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,H,0,0,0,0,0,A,0,0,0,0,0,1,0,0
51,270410,2015091000,2015-09-10 00:00:00,15:00,900.0,1800.0,3600.0,Half1,0,1,NE,PIT,PIT,away,NE,PIT,80.0,1,0,1.0,0.0,PIT 20,10,18,(15:00) De.Williams right tackle to PIT 38 for...,run,18.0,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-09-10 20:40:00,2015.0,0 days 00:00:00.000000000,2015-09-10 20:40:00,2015-09-10 20:40:00,2015-09-10 21:00:00,1010.6,Foxborough,16.1,FieldTurf CORE,87.0,0.3,Open,NE,18.3,2015-09-10 21:00:00,2015-09-11 01:00:00,NaN,30.0,20.5,0,PIT @ NE 2015-09-10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,H,0,0,0,0,0,A,0,0,0,0,0,1,0,0


In [5]:
# Using a form of Joey's agg function
def my_aggfunc(x):
    values = {
        'pass_home':x[x['posteam_type']=='home']['play_type'].eq('pass').sum(),
        'pass_away':x[x['posteam_type']=='away']['play_type'].eq('pass').sum(),
        
        'run_home':x[x['posteam_type']=='home']['play_type'].eq('run').sum(),
        'run_away':x[x['posteam_type']=='away']['play_type'].eq('run').sum(),
        
        'TO_home':x[x['posteam_type']=='home']['INT'].sum() +\
        x[x['posteam_type']=='home']['fumble'].sum(),
        'TO_away':x[x['posteam_type']=='away']['INT'].sum() +\
        x[x['posteam_type']=='away']['fumble'].sum(),
        
        'yards_home':x[(x['posteam_type']=='home') & (x['end_of_drive']==1)]['ydsnet'].sum(),
        'yards_away':x[(x['posteam_type']=='away') & (x['end_of_drive']==1)]['ydsnet'].sum(),
        
        'fg_at_home':x[(x['posteam_type']=='home')&(x['play_type']=='field_goal')]['play_type'].count(),
        'fg_at_away':x[(x['posteam_type']=='away')&(x['play_type']=='field_goal')]['play_type'].count(),
        
        'fg_md_home':x[(x['posteam_type']=='home')&(x['play_type']=='field_goal')]['play_type'].count(),
        'fg_md_away':x[(x['posteam_type']=='away')&(x['play_type']=='field_goal')]['play_type'].count(),
        
        'pass_yards_home':x[(x['posteam_type']=='home')&(x['play_type']=='pass')]['yards_gained'].sum(),
        'pass_yards_away':x[(x['posteam_type']=='away')&(x['play_type']=='pass')]['yards_gained'].sum(),
        
        'run_yards_home':x[(x['posteam_type']=='home')&(x['play_type']=='run')]['yards_gained'].sum(),
        'run_yards_away':x[(x['posteam_type']=='away')&(x['play_type']=='run')]['yards_gained'].sum()
        
    }
    return pd.Series(values)

In [205]:
new_df = pd.DataFrame()
new_df = nfl_small2_end_of_drive.groupby(['game_id']).apply(my_aggfunc)

In [206]:
new_df['home_team'] = nfl_small2_end_of_drive.groupby('game_id')['home_team'].first()
new_df['away_team'] = nfl_small2_end_of_drive.groupby('game_id')['away_team'].first()
new_df['home_score'] = nfl_small2_end_of_drive.groupby('game_id')['total_home_score'].last()
new_df['away_score'] = nfl_small2_end_of_drive.groupby('game_id')['total_away_score'].last()

In [207]:
# Creating DF with rows for each team in every game
test_df = new_df.reset_index()[['game_id','home_team','away_team']]\
                .melt(id_vars='game_id',var_name = 'home_away',value_name="Team")
test_df

,game_id,home_away,Team
0,2015091000,home_team,NE
1,2015091300,home_team,CHI
2,2015091301,home_team,LA
3,2015091302,home_team,JAX
4,2015091303,home_team,WAS
...,...,...,...
1977,2018121608,away_team,TEN
1978,2018121609,away_team,SEA
1979,2018121610,away_team,NE
1980,2018121611,away_team,PHI


In [208]:
test_df['year'] = test_df['game_id'].astype('str').str.extract(r'(^[0-9]{4})')
test_df.head()

,game_id,home_away,Team,year
0,2015091000,home_team,NE,2015
1,2015091300,home_team,CHI,2015
2,2015091301,home_team,LA,2015
3,2015091302,home_team,JAX,2015
4,2015091303,home_team,WAS,2015


In [209]:
test_df['game_id'][:4]

0    2015091000
1    2015091300
2    2015091301
3    2015091302
Name: game_id, dtype: int64

In [210]:
# Merging with team home stats
test_df_home = test_df.merge(new_df.reset_index(),how='inner',left_on=['game_id','Team'],\
                             right_on=['game_id','home_team'])

In [211]:
# Setting variables for Home Team
test_df_home['team_against'] = test_df_home[test_df_home['home_away']=='home_team']['away_team']
test_df_home['score'] = test_df_home[test_df_home['home_away']=='home_team']['home_score']
test_df_home['score_against'] = test_df_home[test_df_home['home_away']=='home_team']['away_score']

test_df_home['yards'] = test_df_home[test_df_home['home_away']=='home_team']['yards_home']
test_df_home['yards_against'] = test_df_home[test_df_home['home_away']=='home_team']['yards_away']

test_df_home['run_plays'] = test_df_home[test_df_home['home_away']=='home_team']['run_home']
test_df_home['run_plays_against'] = test_df_home[test_df_home['home_away']=='home_team']['run_away']

test_df_home['pass_plays'] = test_df_home[test_df_home['home_away']=='home_team']['pass_home']
test_df_home['pass_plays_against'] = test_df_home[test_df_home['home_away']=='home_team']['pass_away']

test_df_home['to_for'] = test_df_home[test_df_home['home_away']=='home_team']['TO_home']
test_df_home['to_against'] = test_df_home[test_df_home['home_away']=='home_team']['TO_away']

test_df_home['fg_at'] = test_df_home[test_df_home['home_away']=='home_team']['fg_at_home']
test_df_home['fg_md'] = test_df_home[test_df_home['home_away']=='home_team']['fg_md_home']

test_df_home['fg_at_against'] = test_df_home[test_df_home['home_away']=='home_team']['fg_at_away']
test_df_home['fg_md_against'] = test_df_home[test_df_home['home_away']=='home_team']['fg_md_away']

test_df_home['pass_yards'] = test_df_home[test_df_home['home_away']=='home_team']['pass_yards_home']
test_df_home['pass_yards_against'] = test_df_home[test_df_home['home_away']=='home_team']['pass_yards_away']

test_df_home['run_yards'] = test_df_home[test_df_home['home_away']=='home_team']['run_yards_home']
test_df_home['run_yards_against'] = test_df_home[test_df_home['home_away']=='home_team']['run_yards_away']

In [212]:
# Creating away only df to merge records from hoem and away team stats
test_df_away = test_df.merge(new_df.reset_index(),how='inner',left_on=['game_id','Team'],right_on=['game_id','away_team'])
test_df_away[test_df_away['game_id']==2017090700]

,game_id,home_away,Team,year,pass_home,pass_away,run_home,run_away,TO_home,TO_away,...,fg_md_home,fg_md_away,pass_yards_home,pass_yards_away,run_yards_home,run_yards_away,home_team,away_team,home_score,away_score
511,2017090700,away_team,KC,2017,39.0,38.0,34.0,24.0,2.0,1.0,...,2.0,0.0,247.0,352.0,125.0,191.0,NE,KC,27,42


In [213]:
# Setting variables for Away Team
test_df_away['team_against'] = test_df_away[test_df_away['home_away']=='away_team']['home_team']
test_df_away['score'] = test_df_away[test_df_away['home_away']=='away_team']['away_score']
test_df_away['score_against'] = test_df_away[test_df_away['home_away']=='away_team']['home_score']

test_df_away['yards'] = test_df_away[test_df_away['home_away']=='away_team']['yards_away']
test_df_away['yards_against'] = test_df_away[test_df_away['home_away']=='away_team']['yards_home']

test_df_away['run_plays'] = test_df_away[test_df_away['home_away']=='away_team']['run_away']
test_df_away['run_plays_against'] = test_df_away[test_df_away['home_away']=='away_team']['run_home']

test_df_away['pass_plays'] = test_df_away[test_df_away['home_away']=='away_team']['pass_away']
test_df_away['pass_plays_against'] = test_df_away[test_df_away['home_away']=='away_team']['pass_home']

test_df_away['to_for'] = test_df_away[test_df_away['home_away']=='away_team']['TO_away']
test_df_away['to_against'] = test_df_away[test_df_away['home_away']=='away_team']['TO_home']

test_df_away['fg_at'] = test_df_away[test_df_away['home_away']=='away_team']['fg_at_away']
test_df_away['fg_md'] = test_df_away[test_df_away['home_away']=='away_team']['fg_md_away']

test_df_away['fg_at_against'] = test_df_away[test_df_away['home_away']=='away_team']['fg_at_home']
test_df_away['fg_md_against'] = test_df_away[test_df_away['home_away']=='away_team']['fg_md_home']

test_df_away['pass_yards'] = test_df_away[test_df_away['home_away']=='away_team']['pass_yards_away']
test_df_away['pass_yards_against'] = test_df_away[test_df_away['home_away']=='away_team']['pass_yards_home']

test_df_away['run_yards'] = test_df_away[test_df_away['home_away']=='away_team']['run_yards_away']
test_df_away['run_yards_against'] = test_df_away[test_df_away['home_away']=='away_team']['run_yards_home']

test_df_away[test_df_away['game_id']==2017090700]

,game_id,home_away,Team,year,pass_home,pass_away,run_home,run_away,TO_home,TO_away,...,to_for,to_against,fg_at,fg_md,fg_at_against,fg_md_against,pass_yards,pass_yards_against,run_yards,run_yards_against
511,2017090700,away_team,KC,2017,39.0,38.0,34.0,24.0,2.0,1.0,...,1.0,2.0,0.0,0.0,2.0,2.0,352.0,247.0,191.0,125.0


In [214]:
new_cols = ['game_id','Team','team_against','score','score_against','yards','yards_against','run_plays',\
            'run_plays_against','pass_plays','pass_plays_against','to_for','to_against','fg_at',\
           'fg_md','fg_at_against','fg_md_against','pass_yards','pass_yards_against','run_yards','run_yards_against']


In [215]:
# Combining home and away dfs
combined_df = test_df_home[new_cols].append(test_df_away[new_cols],ignore_index=True)
combined_df[combined_df['game_id']==2017090700]

,game_id,Team,team_against,score,score_against,yards,yards_against,run_plays,run_plays_against,pass_plays,...,to_for,to_against,fg_at,fg_md,fg_at_against,fg_md_against,pass_yards,pass_yards_against,run_yards,run_yards_against
511,2017090700,NE,KC,27,42,425.0,538.0,34.0,24.0,39.0,...,2.0,1.0,2.0,2.0,0.0,0.0,247.0,352.0,125.0,191.0
1502,2017090700,KC,NE,42,27,538.0,425.0,24.0,34.0,38.0,...,1.0,2.0,0.0,0.0,2.0,2.0,352.0,247.0,191.0,125.0


In [216]:
# Creating my target variable wins
combined_df.loc[combined_df['score'] > combined_df['score_against'],'win'] = 1

combined_df['win'].fillna(0,inplace=True)

In [217]:
# Creating additional variables
combined_df['pass_run_ratio'] = round(combined_df['pass_plays'] / combined_df['run_plays'],2)
combined_df['yard_diff'] = combined_df['yards'] - combined_df['yards_against']
combined_df['to_diff'] = combined_df['to_against'] - combined_df['to_for']

combined_df['fg_rate'] = round(combined_df['fg_md'] / combined_df['fg_at'],2)
combined_df['fg_rate_against'] = round(combined_df['fg_md_against'] / combined_df['fg_at_against'],2)

# Setting fg rates to 50% for teams not attempting field goals
combined_df['fg_rate'].fillna(0.5,inplace=True)
combined_df['fg_rate_against'].fillna(0.5,inplace=True)

combined_df['pass_yds_per_at'] = combined_df['pass_yards'] / combined_df['pass_plays']
combined_df['run_yds_per_at'] = combined_df['run_yards'] / combined_df['run_plays']

combined_df['pass_yds_per_at_against'] = combined_df['pass_yards_against'] / combined_df['pass_plays_against']
combined_df['run_yds_per_at_against'] = combined_df['run_yards_against'] / combined_df['run_plays_against']

In [218]:
# Team wins overall for sorting
team_wins_overall = []
years = [2015,2016,2017,2018]
year_list = []
team_list = []

for team in teams:
    for year in years:
        home_wins = sum(nfl_small2_end_of_drive[(nfl_small2_end_of_drive['end_of_game'] == 1) &\
                                                (nfl_small2_end_of_drive['year'] == year) &\
                                                (nfl_small2_end_of_drive[team] == 'H')
                                               ]['home_team_win'])

        away_wins = sum(nfl_small2_end_of_drive[(nfl_small2_end_of_drive['end_of_game'] == 1) &\
                                                (nfl_small2_end_of_drive['year'] == year) &\
                                                (nfl_small2_end_of_drive[team] == 'A')
                                               ]['away_team_win'])

        # Due to lack of data, normalizing 2018 to 16 week season
        if year == 2018:
            all_wins = np.round((home_wins + away_wins) * 16/14,0)
            team_wins_overall.append(round(all_wins,0))
            year_list.append(year)
            team_list.append(team)
            
        else:
            all_wins = home_wins + away_wins
            team_wins_overall.append(round(all_wins,0))
            year_list.append(year)
            team_list.append(team)
    

team_by_wins = pd.DataFrame({'Team':team_list,'wins':team_wins_overall,'year':year_list}).sort_values(by=['wins'])

# Could change this to mathematical bins
bins = [-1, 7, 9, 17]
labels = ['5 or Less', 'Between 6 and 9','10+ Wins']
team_by_wins['binned'] = pd.cut(team_by_wins['wins'], bins=bins, labels=labels)

In [219]:
team_by_wins[team_by_wins['Team']=='NE']

,Team,wins,year,binned
83,NE,10.0,2018,10+ Wins
80,NE,12.0,2015,10+ Wins
82,NE,13.0,2017,10+ Wins
81,NE,14.0,2016,10+ Wins


In [220]:
combined_df['year'] = test_df['game_id'].astype('str').str.extract(r'(^[0-9]{4})').astype('int')

In [221]:
combined_df = combined_df.merge(team_by_wins[['Team','binned','year']],how='left',on = ['Team','year'])

In [222]:
combined_df.head()

,game_id,Team,team_against,score,score_against,yards,yards_against,run_plays,run_plays_against,pass_plays,...,yard_diff,to_diff,fg_rate,fg_rate_against,pass_yds_per_at,run_yds_per_at,pass_yds_per_at_against,run_yds_per_at_against,year,binned
0,2015091000,NE,PIT,28,21,366.0,429.0,23.0,25.0,34.0,...,-63.0,1.0,0.5,1.0,8.264706,3.565217,7.904762,5.360000,2015,10+ Wins
1,2015091300,CHI,GB,23,31,416.0,336.0,33.0,28.0,36.0,...,80.0,-1.0,1.0,1.0,5.916667,5.727273,8.217391,4.785714,2015,5 or Less
2,2015091301,LA,SEA,33,29,354.0,338.0,26.0,32.0,29.0,...,16.0,-2.0,1.0,1.0,9.517241,2.923077,4.659574,3.968750,2015,5 or Less
3,2015091302,JAX,CAR,9,19,255.0,253.0,21.0,33.0,44.0,...,2.0,-3.0,1.0,1.0,3.840909,4.571429,4.787879,3.242424,2015,5 or Less
4,2015091303,WAS,MIA,10,16,290.0,264.0,36.0,17.0,32.0,...,26.0,-2.0,1.0,1.0,5.875000,4.500000,4.918919,4.411765,2015,Between 6 and 9


In [223]:
combined_df[combined_df['binned'].isnull()]

,game_id,Team,team_against,score,score_against,yards,yards_against,run_plays,run_plays_against,pass_plays,...,yard_diff,to_diff,fg_rate,fg_rate_against,pass_yds_per_at,run_yds_per_at,pass_yds_per_at_against,run_yds_per_at_against,year,binned


In [224]:
# Saving end of game stats to csv
combined_df.to_csv('nfl_end_of_game.csv')